# Data Cleaning - Zillow Data

## Introduction

In the following notebook, I will be cleaning data from [Zillow](https://www.zillow.com/research/data/). This data will be used to supplement my analysis of the Airbnb listings in the San Francisco Area.

I intend to use this data to compare the cost of rent in SF to the cost of renting an average Airbnb for a month.

**Read in necessary libraries**

In [5]:
#Read in libraries
import dask.dataframe as dd
import swifter

import pandas as pd

import re

import numpy as np
from scipy import stats

**Settings for Notebook**

In [6]:
#Increase number of columns and rows displayed by Pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

#supress future warnings
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)

**Read in Data**

In [7]:
#Set path to Zillow Data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\01_Raw\Zillow Raw Data\Zip_MedianRentalPricePerSqft_AllHomes.csv'

#Read in Zillow data
zillow = pd.read_csv(path, header=1)

## Data Preview

In [8]:
#Print shape dtypes of calendar data
print('Original zillow data shape:', zillow.shape)

#Preview Zillow data
zillow.head()

Original zillow data shape: (6647, 123)


,RegionName,City,State,Metro,CountyName,SizeRank,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10
0,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.76058583563554,4.75926225216817,4.72836850649351,4.64949307541222,4.59623430962343,4.50281425891182,4.50281425891182,4.56317204301075,4.52049184484927,4.60216891910046,4.57918682840643,4.60268948655257,4.75888324873096,4.74417376722262,4.74500577007049,4.64285714285714,4.72461414875685,4.77064220183486,4.78945035460993,4.71783655135425,4.75638994265917,4.79683972911964,4.87218045112782,4.81597038398057,4.81729197601206,4.84934472934473,4.85578965282505,4.88888888888889,4.88431876606684,4.75974025974026,4.84488975992161,4.85626373626374,4.85976408912189,4.82315112540193,4.78151921802518,4.86759581881533,4.75926225216817,4.85626373626374,4.91803278688525,4.81831395348837,4.85538461538462,4.82815057283142,4.83857949959645,4.82274902131361,4.85538461538462,4.83058813526497,4.84988452655889,4.85714285714286,4.85714285714286,4.88251470430332,4.85714285714286,4.85443223443223,4.62277331470485,4.58981229127481,4.63517409625194,4.69043151969981,4.68439704639961,4.71236247828605,4.60216891910046,4.70994790270705,4.72727272727273,4.74137931034483,4.84530301829993,4.74285342704264,4.75964125560538,4.79166666666667,4.79583333333333,4.75888324873096,4.77821282251545,4.73678137340376,4.71544715447155,4.75926225216817,4.79166666666667
1,10023,New York,NY,New York-Newark-Jersey City,New York County,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00853816012112,5.1244167962675,5.196,5.22210357210357,5.28953229398664,5.21428571428571,5.2871826926416,5.35114465476605,5.5,5.45755237045204,5.46099290780142,5.48961424332344,5.49480712166172,5.45255949114999,5.46099290780142,5.43564572982298,5.39325842696629,5.39325842696629,5.44722417062843,5.50358324145535,5.5,5.57963163596966,5.52411657831492,5.47795165718916,5.47645125958379,5.56328233657858,5.56328233657858,5.5,5.49450549450549,5.5,5.46099290780142,5.41125541125541,5.41272570937231,5.41666666666667,5.37636761487965,5.35294117647059,5.33768656716418,5.26853718637993,5.06445672191529,5.41907514450867,5.41760299625468,5.37636761487965,5.50280898876404,5.52238805970149,5.45901639344262,5.45901639344262,5.43929795202805,5.37117903930131,5.41041440135315,5.34303416074707,5.34303416074707,5.37636761487965,5.13106433033714,5.07368421052632,5.23076923076923,5.31111441995897,5.36965305736835,5.33333333333333,5.18933784222572,5.20385050962627,5.34402852049911,5.46099290780142,5.35294117647059,5.37636761487965,5.24433849821216,5.23936525506212,5.35489285296069,5.37636761487965,5.34164304589811,5.34124629080119,5.39351851851852,5.33333333333333,5.32819932914406
2,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

**Convert zillow data into a tidy dataset**

In [9]:
#Set columns for melt
id_vars = list(zillow.loc[:,:'SizeRank'].columns.values)
value_vars = list(zillow.iloc[:,6:].columns.values)

#Melt zillow. Create a Data and Price/SqrFt column
zillow = zillow.melt(id_vars= id_vars,value_vars= value_vars, var_name='Date', value_name= 'Price_SqrFt' )

#Print updated shape and dtypes of zillow data
print('Updated zillow data shape:',zillow.shape)
print('Zillow data types: \n', zillow.dtypes)

Updated zillow data shape: (777699, 8)
Zillow data types: 
 RegionName     object
City           object
State          object
Metro          object
CountyName     object
SizeRank       object
Date           object
Price_SqrFt    object
dtype: object


There appears to be some string values in RegionName, SizeRank, Date, and Price_SqrFt that we will need to isolate and remove          

In [10]:
#Pull rows that contain string in region name and remove
zillow.drop(zillow[zillow['RegionName'].str.contains('[A-Za-z]')].index,inplace=True)

#Create list of cols to convert into numeric
cols = ['RegionName', 'SizeRank', 'Price_SqrFt']

#Set data types
zillow.Date= zillow.Date.astype('datetime64[ns]')
zillow[cols] = zillow[cols].apply(pd.to_numeric, errors='coerce')

In [11]:
#Print updated shape and dtypes of zillow data
print('Updated zillow data shape:',zillow.shape)
print('Zillow data types: \n', zillow.dtypes)

Updated zillow data shape: (777465, 8)
Zillow data types: 
 RegionName            float64
City                   object
State                  object
Metro                  object
CountyName             object
SizeRank              float64
Date           datetime64[ns]
Price_SqrFt           float64
dtype: object


**Zillow Metrics**

In [12]:
#Describe zillow
display(zillow.describe())

,RegionName,SizeRank,Price_SqrFt
count,777348.000000,777348.000000,385366.000000
mean,55429.895846,1661.414509,1.299695
std,29615.932146,958.969772,1.037336
min,1545.000000,1.000000,0.111453
25%,30329.000000,831.000000,0.808539
50%,52245.500000,1661.500000,1.007005
75%,85013.000000,2492.000000,1.430175
max,99705.000000,3322.000000,23.969319


In [13]:
#Zillow variance
print('Variance:\n', zillow.var(axis=0))

Variance:
 RegionName     8.771034e+08
SizeRank       9.196230e+05
Price_SqrFt    1.076066e+00
dtype: float64


# Export Cleaned Data

In [14]:
#Set path to export cleaned zillow data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate\12_23_2019_Zillow_Cleaned.csv'

#Write file
zillow.to_csv(path, sep=',')